In [73]:
import json

In [74]:
# Метод для чтения лемм из файла в словарь
def read_lemmas(lemmas_file_path):
    lemmas_dict = {}
    with open(lemmas_file_path, 'r') as lemmas_file:
        for lemma_line in lemmas_file:
            lemma_with_tokens = lemma_line.split(':')
            lemma = lemma_with_tokens[0].strip()
            tokens = [token.strip() for token in lemma_with_tokens[1].split(' ') if token.strip() != '']
            lemmas_dict[lemma] = tokens
    return lemmas_dict

In [75]:
import os

data_dir_path = '../Выкачка/'
data_file_pattern = 'выкачка_'

# Метод получения индекса файла из названия формата 'file_pattern_{index}.txt'
def get_file_index(filename):
    return int(filename[len(data_file_pattern):].replace('.txt', ''))

# Метод создания инвертированного индекса на основе пары лемма - токены
def count_inverted_index(lemma, words):
    # Индексы файлов, которые связаны с переданной леммой
    files_indices = []
    # Проходим по всем файлам выкачек
    for filename in os.listdir(data_dir_path):
        if data_file_pattern not in filename:
            continue
        file_path = data_dir_path + filename
        file_index = get_file_index(filename)
        with open(file_path, 'r') as text_file:
            # Ищем, есть ли в файле есть слово, связанное с леммой
            file_data = text_file.read().lower()
            if any(word in file_data for word in words):
                file_index = get_file_index(filename)
                files_indices.append(file_index)
    # Сортируем полученные индексы файлов и отдаем в виде словаря лемма - индексы файлов
    files_indices.sort()
    return {lemma : files_indices}

In [76]:
# Метод для склеивания двух словарей
def merge_dicts(dict1, dict2):
    for key in dict2:
        if key in dict1:
            dict1[key] = dict1[key] + dict2[key]
        else:
            dict1[key] = dict2[key]
    return dict1

In [77]:
# Метод записи инвертированного индекса в файл в json формате
def write_json_indices(file_path, indices):
    sorted_indices_keys = sorted(indices.keys())
    with open(file_path, 'w') as indices_file:
        for index_key in sorted_indices_keys:
            index_documents = indices[index_key]
            index_line = {"count": len(index_documents), "inverted_array": index_documents, "word": index_key}
            index_line_json = json.dumps(index_line)
            indices_file.write(index_line_json + '\n') 

In [83]:
# Метод построения полного инвертированного индекса:
# cтроим индекс для каждой леммы и затем склеиваем полученные словари
# результат записывается в файл inverted_index.txt
lemmas = read_lemmas('../lemmas.txt')
inverted_indices = {}
for lemma in lemmas:
    lemma_inverted_index = count_inverted_index(lemma, lemmas[lemma])
    inverted_indices = merge_dicts(inverted_indices, lemma_inverted_index)
write_json_indices('../inverted_index.txt', inverted_indices)

# Булев поиск

In [79]:
# Метод получения инвертированного списка индексов из файла в формат словаря
def get_inverted_index_dict():
    inverted_index_dict = {}
    with open('../inverted_index.txt', 'r') as index_file:
        for line in index_file:
            data = json.loads(line)
            inverted_index_dict[data['word']] = data['inverted_array']
    return inverted_index_dict

# Метод получения ссылок на статьи на основе ее индекса
def get_page_links_by_indices(index_list):
    pages = []
    with open('../index.txt', 'r') as pages_file:
        for line in pages_file:
            page_index_with_link = line.split(' ')
            index = int(page_index_with_link[0])
            if index in index_list:
                pages.append(page_index_with_link[1].replace('\n', ''))
    return pages

In [80]:
# Метод поиска, на вход подается строка в формате (слово (&/| слово){1-...})
def search(text):
    search_params = text.strip().split(' ')
    # Если в поиске нет слов, то получаем пустой список статей
    if len(search_params) < 1:
        return []
    inverted_indices = get_inverted_index_dict()
    page_indices = []
    if search_params[0] in inverted_indices:
        page_indices = inverted_indices[search_params[0]]
    # Для каждой пары оператор - слово находим на основе инвертированного списка индексов нужные индексы статей и используем пересечение/объединение
    for i in range(1, len(search_params), 2):
        operator = search_params[i]
        search_word = search_params[i + 1]
        search_word_page_indices = []
        if search_word in inverted_indices:
            search_word_page_indices = inverted_indices[search_word]
        if operator == '&':
            page_indices = list(set(page_indices) & set(search_word_page_indices))
        elif operator == '|':
            page_indices = list(set(page_indices) | set(search_word_page_indices))
    return get_page_links_by_indices(set(page_indices))

In [81]:
search('work & teacher | find | treasure')

['https://www.imdb.com/title/tt11934846/',
 'https://www.imdb.com/title/tt9389998/',
 'https://www.imdb.com/title/tt15097216/',
 'https://www.imdb.com/title/tt9766332/',
 'https://www.imdb.com/title/tt0986264/',
 'https://www.imdb.com/title/tt8772296/',
 'https://www.imdb.com/title/tt1464335/',
 'https://www.imdb.com/title/tt9288030/',
 'https://www.imdb.com/title/tt1520211/',
 'https://www.imdb.com/title/tt5788792/',
 'https://www.imdb.com/title/tt11252248/',
 'https://www.imdb.com/title/tt0903747/',
 'https://www.imdb.com/title/tt10944760/',
 'https://www.imdb.com/title/tt0119558/',
 'https://www.imdb.com/title/tt2278871/',
 'https://www.imdb.com/title/tt12404772/',
 'https://www.imdb.com/title/tt11686490/',
 'https://www.imdb.com/title/tt0211915/',
 'https://www.imdb.com/title/tt1879030/',
 'https://www.imdb.com/title/tt0107977/',
 'https://www.imdb.com/title/tt2872732/',
 'https://www.imdb.com/title/tt2333784/',
 'https://www.imdb.com/title/tt9738784/',
 'https://www.imdb.com/title

In [82]:
search('work | teacher & find | treasure')

['https://www.imdb.com/title/tt11934846/',
 'https://www.imdb.com/title/tt9389998/',
 'https://www.imdb.com/title/tt15097216/',
 'https://www.imdb.com/title/tt0986264/',
 'https://www.imdb.com/title/tt1464335/',
 'https://www.imdb.com/title/tt0211915/',
 'https://www.imdb.com/title/tt10525672/']